In [98]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from io import StringIO

In [99]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
data = requests.get(standings_url)

In [ ]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]


IndexError: list index out of range

In [101]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [60]:
# Step 3: Define years and loop through them
years = list(range(2024, 2020, -1))
all_matches = []


In [61]:
from io import StringIO
print(StringIO)

<class '_io.StringIO'>


In [62]:
from io import StringIO
import pandas as pd

html_data = StringIO("<table><tr><th>Scores & Fixtures</th></tr></table>")
matches = pd.read_html(html_data, match="Scores & Fixtures")[0]
print(matches)


Empty DataFrame
Columns: [Scores & Fixtures]
Index: []


In [63]:


soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]



In [65]:
data = requests.get("https://fbref.com/de/squads/822bd0ba/2024-2025/matchlogs/all_comps/shooting/Liverpool-Match-Logs-All-Competitions")

In [66]:
shooting = pd.read_html(data.text, match=("Shooting")[0])

/tmp/ipykernel_429243/31653919.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match=("Shooting")[0])


In [67]:

# Access the first DataFrame
shooting_df = shooting[0]

# View the first rows
print(shooting_df.head())


  Für Liverpool                                                                \
          Datum Uhrzeit            Wett         Runde  Tag  Spielort Ergebnis   
0    17.08.2024   12:30  Premier League  Spielwoche 1  Sa.  Auswärts        S   
1    25.08.2024   16:30  Premier League  Spielwoche 2  So.      Heim        S   
2    01.09.2024   16:00  Premier League  Spielwoche 3  So.  Auswärts        S   
3    14.09.2024   15:00  Premier League  Spielwoche 4  Sa.      Heim        N   
4    17.09.2024   21:00    Champions Lg     Ligaphase  Di.  Auswärts        S   

                              ... Standard                Erwartet       \
    Tf   Tk           Gegner  ...    Entf.   FS Elf VeElf       xG npxG   
0  2.0  0.0     Ipswich Town  ...    148.0  0.0   0     0      2.6  2.6   
1  2.0  0.0        Brentford  ...    136.0  1.0   0     0      2.5  2.5   
2  3.0  0.0   Manchester Utd  ...    134.0  0.0   0     0      1.8  1.8   
3  0.0  1.0  Nott'ham Forest  ...    149.0  0.0   0     0

In [68]:
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text, 'html.parser')
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a') if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    # Get the URL for the previous season
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        
        # Get match data
        try:
            matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        except ValueError:
            continue
        
        soup = BeautifulSoup(data.text, 'html.parser')
        links = [l.get("href") for l in soup.find_all('a') if l and 'all_comps/shooting/' in l]
        
        if links:
            shooting_url = f"https://fbref.com{links[0]}"
            data = requests.get(shooting_url)
            
            try:
                shooting = pd.read_html(data.text, match="Shooting")[0]
                shooting.columns = shooting.columns.droplevel()
                
                # Merge match data with shooting data
                team_data = matches.merge(
                    shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]],
                    on="Date"
                )
            except (ValueError, KeyError):
                continue
            
            # Filter for Premier League matches
            team_data = team_data[team_data["Comp"] == "Premier League"]
            team_data["Season"] = year
            team_data["Team"] = team_name
            all_matches.append(team_data)
        
        time.sleep(1)  # Be polite with requests

# Step 4: Combine all data and save to CSV
if all_matches:
    match_df = pd.concat(all_matches)
    match_df.columns = [c.lower() for c in match_df.columns]
    match_df.to_csv("matches.csv", index=False)
    print("Data saved to matches.csv")
else:
    print("No match data collected.")

No match data collected.


In [72]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from io import StringIO

# Step 1: Fetch initial standings page
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
data = requests.get(standings_url)
soup = BeautifulSoup(data.text, 'html.parser')

# Step 2: Get team URLs
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links if l and '/squads/' in l]
team_urls = [f"https://fbref.com{l}" for l in links]

# Step 3: Define years and loop through them
years = list(range(2024, 2020, -1))
all_matches = []

for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text, 'html.parser')
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a') if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    # Get the URL for the previous season
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        
        # Get match data
        try:
            matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        except ValueError:
            continue
        
        soup = BeautifulSoup(data.text, 'html.parser')
        links = [l.get("href") for l in soup.find_all('a') if l and 'all_comps/shooting/' in l]
        
        if links:
            shooting_url = f"https://fbref.com{links[0]}"
            data = requests.get(shooting_url)
            
            try:
                shooting = pd.read_html(data.text, match="Shooting")[0]
                shooting.columns = shooting.columns.droplevel()
                
                # Merge match data with shooting data
                team_data = matches.merge(
                    shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]],
                    on="Date"
                )
            except (ValueError, KeyError):
                continue
            
            # Filter for Premier League matches
            team_data = team_data[team_data["Comp"] == "Premier League"]
            team_data["Season"] = year
            team_data["Team"] = team_name
            all_matches.append(team_data)
        
        time.sleep(1)  # Be polite with requests

# Step 4: Combine all data and save to CSV
if all_matches:
    match_df = pd.concat(all_matches)
    match_df.columns = [c.lower() for c in match_df.columns]
    match_df.to_csv("matches.csv", index=False)
    print("Data saved to matches.csv")
else:
    print("No match data collected.")


No match data collected.


In [70]:
print(len(soup.select('table.stats_table')))


24


In [95]:
print(standings_table.prettify())


<table class="stats_table sortable min_width force_mobilize" data-cols-to-freeze=",2" id="results2020-202191_overall">
 <caption>
  Premier League Table
 </caption>
 <colgroup>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
 </colgroup>
 <thead>
  <tr>
   <th aria-label="Rank" class="poptip sort_default_asc center" data-stat="rank" data-tip="&lt;strong&gt;Rank&lt;/strong&gt;&lt;br&gt;Squad finish in competition&lt;br&gt;Finish within the league or competition.&lt;br&gt;For knockout competitions may show final round reached.&lt;br&gt;Colors and arrows represent promotion/relegation or qualifiation for continental cups.&lt;br&gt;Trophy indicates team won league whether by playoffs or by leading the table.&lt;br&gt;Star indicates topped table in league USING another means of naming champion." scope="col">
    Rk
   </th>
   <th aria-label="Squad" class="poptip sort_d

In [74]:
soup = BeautifulSoup(data.text, 'html.parser')
links = [l.get("href") for l in soup.find_all('a') if l and 'all_comps/shooting/' in l]
print("Shooting Links:", links)


Shooting Links: []


In [75]:
print("Standings URL:", standings_url)


Standings URL: https://fbref.com/en/comps/9/2020-2021/2020-2021-Premier-League-Stats


In [73]:
links = [l.get("href") for l in standings_table.find_all('a') if '/squads/' in l]
print("Extracted Links:", links)


Extracted Links: []


In [77]:
from bs4 import BeautifulSoup
import requests

standings_url = "https://fbref.com/en/comps/9/2020-2021/2020-2021-Premier-League-Stats"
data = requests.get(standings_url)

soup = BeautifulSoup(data.text, 'html.parser')

# Print each table to find the one with teams
for i, table in enumerate(soup.select('table.stats_table')):
    print(f"Table {i}:")
    print(table.prettify())
    print("=" * 80)


Table 0:
<table class="stats_table sortable min_width force_mobilize" data-cols-to-freeze=",2" id="results2020-202191_overall">
 <caption>
  Premier League Table
 </caption>
 <colgroup>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
  <col/>
 </colgroup>
 <thead>
  <tr>
   <th aria-label="Rank" class="poptip sort_default_asc center" data-stat="rank" data-tip="&lt;strong&gt;Rank&lt;/strong&gt;&lt;br&gt;Squad finish in competition&lt;br&gt;Finish within the league or competition.&lt;br&gt;For knockout competitions may show final round reached.&lt;br&gt;Colors and arrows represent promotion/relegation or qualifiation for continental cups.&lt;br&gt;Trophy indicates team won league whether by playoffs or by leading the table.&lt;br&gt;Star indicates topped table in league USING another means of naming champion." scope="col">
    Rk
   </th>
   <th aria-label="Squad" class="popt

In [80]:
standings_table = soup.select('table.stats_table')[1]
links = standings_table.find_all('a', href=True)
links = [l.get("href") for l in links if '/squads/' in l]

print("Extracted Links:", links)


Extracted Links: []


In [93]:
years = list(range(2024, 2020, -1))
all_matches = []
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [96]:

for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)

IndexError: list index out of range

In [83]:
match_df = pd.concat(all_matches)

In [85]:
match_df.columns = [c.lower() for c in match_df.columns]

In [86]:
match_df.to_csv("matches.csv")